first download the .csvs 

curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt > '9_3_2016_turnstiles.csv'  
curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt > '9_10_2016_turnstiles.csv'  
curl http://web.mta.info/developers/data/nyct/turnstile/turnstile_160917.txt > '9_17_2016_turnstiles.csv'  


Read in the .csvs and create one dataframe containing all of them.

In [129]:
import pandas as pd

sep_03_df = pd.read_csv('9_3_2016_turnstiles.csv')
sep_10_df = pd.read_csv('9_10_2016_turnstiles.csv')
sep_17_df = pd.read_csv('9_17_2016_turnstiles.csv')

df = sep_03_df
df.append(sep_10_df)
df.append(sep_17_df)
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456,BMT,09/10/2016,00:00:00,REGULAR,5817505,1971936
1,A002,R051,02-00-00,59 ST,NQR456,BMT,09/10/2016,04:00:00,REGULAR,5817544,1971940
2,A002,R051,02-00-00,59 ST,NQR456,BMT,09/10/2016,08:00:00,REGULAR,5817564,1971981
3,A002,R051,02-00-00,59 ST,NQR456,BMT,09/10/2016,12:00:00,REGULAR,5817678,1972094
4,A002,R051,02-00-00,59 ST,NQR456,BMT,09/10/2016,16:00:00,REGULAR,5817925,1972180


Create 'STATID', 'TIMESTAMP', 'DIFFERENCE' columns.
STATID is a unique identifier for a station using the station's name and the train lines that can be accessed there.
TIMESTAMP is a combination of the DATE and TIME fields.
DIFFERENCE is a count of the number of entries that occured since the last recorded TIMESTAMP.

In [93]:
df['STATID'] = df.STATION + df.LINENAME

In [94]:
df['TIMESTAMP'] = pd.to_datetime((df.DATE + ' ' + df.TIME), format='%m/%d/%Y %H:%M:%S')

In [95]:
df = df.sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])
df['DIFFERENCE']=df.groupby(['STATID','UNIT','SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

In [96]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
29628,H007,R248,00-00-00,1 AV,L,BMT,09/10/2016,00:00:00,REGULAR,11490942,12581032,1 AVL,2016-09-10 00:00:00,NaN
29629,H007,R248,00-00-00,1 AV,L,BMT,09/10/2016,04:00:00,REGULAR,11491053,12581378,1 AVL,2016-09-10 04:00:00,111
29630,H007,R248,00-00-00,1 AV,L,BMT,09/10/2016,08:00:00,REGULAR,11491154,12581693,1 AVL,2016-09-10 08:00:00,101
29631,H007,R248,00-00-00,1 AV,L,BMT,09/10/2016,12:00:00,REGULAR,11491718,12582439,1 AVL,2016-09-10 12:00:00,564
29632,H007,R248,00-00-00,1 AV,L,BMT,09/10/2016,16:00:00,REGULAR,11492521,12583529,1 AVL,2016-09-10 16:00:00,803


Now look for outliers in the DIFFERENCE data.

In [107]:
sorted_df = df.sort_values('DIFFERENCE')
sorted_df.DIFFERENCE.head(25)

1621     -1894869117
159387     -74671740
2598       -66849133
187611     -12497846
148910      -6130945
99821       -3664512
99321       -3252754
34707       -1054560
129526      -1014544
34749        -761957
34839        -227132
34887        -197042
115460        -36263
34797         -29042
34525         -24360
34570         -20866
34665         -10817
11066          -1959
11060          -1861
11048          -1850
185451         -1848
185457         -1830
3476           -1812
185445         -1800
185439         -1761
Name: DIFFERENCE, dtype: float64

Without any more investigation it seem reasonable to remove the first 12 entries. 

In [117]:
sorted_df = sorted_df[sorted_df['DIFFERENCE'] > -197042]

In [118]:
sorted_df.DIFFERENCE.head(5)

115460   -36263
34797    -29042
34525    -24360
34570    -20866
34665    -10817
Name: DIFFERENCE, dtype: float64

In [119]:
sorted_df[sorted_df['DIFFERENCE'] == -36263]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
115460,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,09:18:36,REGULAR,0,0,LACKAWANNA1,2016-09-14 09:18:36,-36263


In [120]:
sorted_df[sorted_df['DATE'] == '09/14/2016'][sorted_df['UNIT'] == 'R550'][sorted_df['SCP'] == '01-02-03'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
115458,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,01:00:25,REGULAR,36263,226248,LACKAWANNA1,2016-09-14 01:00:25,0
115459,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,05:12:25,REGULAR,36263,226248,LACKAWANNA1,2016-09-14 05:12:25,0
115460,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,09:18:36,REGULAR,0,0,LACKAWANNA1,2016-09-14 09:18:36,-36263
115461,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,13:30:35,REGULAR,1,2,LACKAWANNA1,2016-09-14 13:30:35,1
115462,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,17:42:35,REGULAR,1,145,LACKAWANNA1,2016-09-14 17:42:35,0
115463,PTH16,R550,01-02-03,LACKAWANNA,1,PTH,09/14/2016,22:08:35,REGULAR,1,312,LACKAWANNA1,2016-09-14 22:08:35,0


In [121]:
sorted_df.DIFFERENCE.head(5)

115460   -36263
34797    -29042
34525    -24360
34570    -20866
34665    -10817
Name: DIFFERENCE, dtype: float64

In [122]:
sorted_df[sorted_df['DIFFERENCE'] == -10817]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
34665,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,13:00:00,REGULAR,38,60,HOWARD BCH JFKA,2016-09-13 13:00:00,-10817


In [124]:
sorted_df[sorted_df['DATE'] == '09/13/2016'][sorted_df['UNIT'] == 'R535'][sorted_df['SCP'] == '01-00-01'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
34662,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,01:00:00,REGULAR,10776,3711,HOWARD BCH JFKA,2016-09-13 01:00:00,36
34663,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,05:00:00,REGULAR,10806,3713,HOWARD BCH JFKA,2016-09-13 05:00:00,30
34664,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,09:00:00,REGULAR,10855,3730,HOWARD BCH JFKA,2016-09-13 09:00:00,49
34665,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,13:00:00,REGULAR,38,60,HOWARD BCH JFKA,2016-09-13 13:00:00,-10817
34666,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,17:00:00,REGULAR,69,134,HOWARD BCH JFKA,2016-09-13 17:00:00,31
34667,JFK02,R535,01-00-01,HOWARD BCH JFK,A,IND,09/13/2016,21:00:00,REGULAR,69,134,HOWARD BCH JFKA,2016-09-13 21:00:00,0


In [125]:
sorted_df.DIFFERENCE.head(50)

115460   -36263
34797    -29042
34525    -24360
34570    -20866
34665    -10817
11066     -1959
11060     -1861
11048     -1850
185451    -1848
185457    -1830
3476      -1812
185445    -1800
185439    -1761
3464      -1755
3470      -1729
11054     -1718
185463    -1699
11042     -1695
95670     -1690
3452      -1620
3458      -1565
11065     -1549
75230     -1546
50209     -1523
75224     -1497
95676     -1495
95682     -1484
1451      -1468
105361    -1460
95664     -1446
105349    -1442
50215     -1428
75236     -1425
50221     -1423
75218     -1415
105355    -1399
95688     -1398
1664      -1397
1670      -1364
127550    -1356
1457      -1354
154546    -1350
1445      -1340
154564    -1333
50227     -1333
75243     -1332
154558    -1331
154536    -1331
105343    -1329
127556    -1317
Name: DIFFERENCE, dtype: float64

In [126]:
sorted_df[sorted_df['DIFFERENCE'] == -1317]

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
127556,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,20:00:00,REGULAR,1894050276,1427149989,TIMES SQ-42 ST1237ACENQRS,2016-09-15 20:00:00,-1317


In [127]:
sorted_df[sorted_df['DATE'] == '09/15/2016'][sorted_df['UNIT'] == 'R033'][sorted_df['SCP'] == '01-00-01'].sort_values(['STATID', 'UNIT','SCP', 'TIMESTAMP'])

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,STATID,TIMESTAMP,DIFFERENCE
127551,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,00:00:00,REGULAR,1894052319,1427148816,TIMES SQ-42 ST1237ACENQRS,2016-09-15 00:00:00,-400
127552,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,04:00:00,REGULAR,1894052254,1427148834,TIMES SQ-42 ST1237ACENQRS,2016-09-15 04:00:00,-65
127553,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,08:00:00,REGULAR,1894052211,1427148949,TIMES SQ-42 ST1237ACENQRS,2016-09-15 08:00:00,-43
127554,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,12:00:00,REGULAR,1894051997,1427149429,TIMES SQ-42 ST1237ACENQRS,2016-09-15 12:00:00,-214
127555,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,16:00:00,REGULAR,1894051593,1427149666,TIMES SQ-42 ST1237ACENQRS,2016-09-15 16:00:00,-404
127556,R148,R033,01-00-01,TIMES SQ-42 ST,1237ACENQRS,IRT,09/15/2016,20:00:00,REGULAR,1894050276,1427149989,TIMES SQ-42 ST1237ACENQRS,2016-09-15 20:00:00,-1317


For the less extreme negative values, it looks as though the counter is working, however it is counting in reverse.
Because there is a drastic jump in the DIFFERENCE values between -10817 and -1959, we will split the data here. Anything less than -10817 will be ignored and the rest will be replaced by their absolute value and included in the count. 

In [128]:
sorted_df = sorted_df[sorted_df['DIFFERENCE'] > -10817]